**SQP**

In [43]:
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def gradient1(f, x, h=1e-4):
    n = len(x)
    grad = np.zeros(n)
    for i in range(n):
        x_forward = np.copy(x)
        x_backward = np.copy(x)
        x_forward[i] += h
        x_backward[i] -= h
        grad[i] = (f(x_forward,[0,0]) - f(x_backward,[0,0])) / (2 * h)
        grad = grad.reshape(-1, 1)
    return np.round(grad, decimals=6)

def gradient2(f, x, h=1e-4):
    n = len(x)
    grad = np.zeros(n)
    for i in range(n):
        x_forward = np.copy(x)
        x_backward = np.copy(x)
        x_forward[i] += h
        x_backward[i] -= h
        grad[i] = (f(x_forward,[0,0]) - f(x_backward,[0,0])) / (2 * h)
    return np.round(grad, decimals=6)


def hessian(f, x, lamda, h=1e-4):
    n = len(x)  # Number of variables (in this case, 2)
    H = np.zeros((n, n))  # Initialize the Hessian matrix with zeros
    for i in range(n):
        for j in range(n):
            if i == j:
                # Diagonal elements: Second partial derivatives w.r.t. the same variable
                x_ij_forward = np.copy(x)
                x_ij_backward = np.copy(x)
                x_ij_forward[i] += h
                x_ij_backward[i] -= h
                f_ij_forward = f(x_ij_forward, lamda)
                f_ij_backward = f(x_ij_backward, lamda)
                H[i, j] = (f_ij_forward - 2*f(x, lamda) + f_ij_backward) / (h**2)

            else:
                # Off-diagonal elements: Mixed partial derivatives
               x_ij_forward = np.copy(x)
               x_ij_backward = np.copy(x)
               x_ij_forward[i] += h
               x_ij_forward[j] += h
               x_ij_backward[i] -= h
               x_ij_backward[j] -= h
               f_ij_forward = f(x_ij_forward, lamda)
               f_ij_backward = f(x_ij_backward, lamda)

               H[i, j] = (f_ij_forward - f((x_ij_forward - 2*h*np.eye(n)[i]), lamda) -
                           f((x_ij_forward - 2*h*np.eye(n)[j]), lamda) + f_ij_backward) / (4 * h**2)
    return np.round(H, decimals=6)

In [44]:
#ECQP
def ECQP(G, d, A_k, b_k, x):
  i = A_k.shape[0]
  Z = np.zeros((i,i))
  L1 = np.hstack((G, -A_k.T))
  L2 = np.vstack((L1, np.hstack((A_k, Z))))

  R1 = -(np.dot(G, x) + d)
  R2 = b_k - np.dot(A_k, x)
  R3 = np.vstack((R1, R2))

  result = np.linalg.pinv(L2) @ R3
  n = G.shape[0]

  p = result[:n]
  p = np.round(p, 7)
  lamda = result[n:]

  return p , lamda

#alpha
def min_alpha(A, b, p, x):
  n = len(b)
  alpha_vec = np.zeros(n)
  for i in range(n):
    if np.dot(A[i], p) < 0:
      alpha_vec[i] = (b[i] - np.dot(A[i], x))/np.dot(A[i], p)
    else:
      alpha_vec[i] = np.inf

  alpha_1 = np.min(alpha_vec)
  alpha = min(1, alpha_1)

  return alpha_vec, alpha

In [45]:
#defining lagrangian for the given problem

def f(x, lamda):
    return 3*x[0]**2 - 4*x[1] - lamda[0]*(2*x[0] + x[1] - 4) - lamda[1]*(-x[0]**2 - x[1]**2 + 37)

def c_e(x, dummy):
  return 2*x[0] + x[1] - 4

def c_i(x, dummy):
  return -x[0]**2 - x[1]**2 + 37

In [46]:
print('\033[1;33mSUBPROBLEM 1\033[0m')
print()
x_0 = np.array([1.0, 2.0])
lamda_0 = np.array([0.0, 0.0])
W_0 = hessian(f,x_0,lamda_0)
d_0 = gradient1(f, x_0)
A_eq_0 = gradient2(c_e, x_0)
b_eq_0 = np.array(-c_e(x_0, 0))
A_iq_0 = gradient2(c_i, x_0)
b_iq_0 = np.array(-c_i(x_0, 0))

# Iteration 1, equality in working set
print('Iteration 1')
print()
p_01 = np.array([[0.0],
                [0.0]])
A_k01 = np.array([A_eq_0])
b_k01 = np.array([b_eq_0])

q01, lamda01 = ECQP(W_0, d_0, A_k01, b_k01, p_01)

print(f'q01:\n {q01}')
print()
print(f'lamda01:\n {lamda01}')
print()
print(f'q01 norm:\n{np.linalg.norm(q01)}')
print()

##norm of qis not zero, need to find alpha
A_01 = np.array([A_iq_0])
b_01 = np.array([b_iq_0])

alpha_vec01, alpha01 = min_alpha(A_01, b_01, q01, p_01)
print(f'alpha_max: \n{alpha01}')
print()
print(f'alpha_vec: \n{alpha_vec01}')
print()

p_02 = p_01 + alpha01*q01


# Iteration 2, equality in working set
print('Iteration 2')
print()

A_k02 = np.array([A_eq_0])
b_k02 = np.array([b_eq_0])

q02, lamda02 = ECQP(W_0, d_0, A_k02, b_k02, p_02)

print(f'q02:\n {q02}')
print()
print(f'lamda02:\n {lamda02}')
print()
print(f'q02 norm:\n{np.linalg.norm(q02)}')
print()

x_0 = x_0.reshape(-1, 1)
x_1 = x_0 + p_02
print(f'x1:\n{x_1}')
print()


print('\033[1;33mSUBPROBLEM 2\033[0m')
print()
x_1 = x_1.flatten()
lamda_1 = np.array([-4.0, 0.0])  #lamda is associated with equality only in previous problem
W_1 = hessian(f,x_1,lamda_1)
d_1 = gradient1(f, x_1)
A_eq_1 = gradient2(c_e, x_1)
b_eq_1 = np.array(-c_e(x_1, 0))
A_iq_1 = gradient2(c_i, x_1)
b_iq_1 = np.array(-c_i(x_1, 0))

# Iteration 1, equality and inequality in working set
print('Iteration 1')
print()
p_11 = np.array([[0.0],
                [0.0]])

A_k11 = np.vstack((A_eq_1, A_iq_1))
b_k11 = np.vstack((b_eq_1, b_iq_1))

q11, lamda11 = ECQP(W_1, d_1, A_k11, b_k11, p_11)

print(f'q11:\n {q11}')
print()
print(f'lamda11:\n {lamda11}')
print()
print(f'q11 norm:\n{np.linalg.norm(q11)}')
print()

p_12 = p_11 + q11 #alpha is 1 since no inactive working set

# Iteration 2, equality and inequality in working set
print('Iteration 2')
print()

A_k12 = np.vstack((A_eq_1, A_iq_1))
b_k12 = np.vstack((b_eq_1, b_iq_1))

q12, lamda12 = ECQP(W_1, d_1, A_k12, b_k12, p_12)
print(f'q12:\n {q12}')
print()
print(f'lamda12:\n {lamda12}')
print()
print(f'q12 norm:\n{np.linalg.norm(q12)}')
print()

x_1 = x_1.reshape(-1, 1)
x_2 = x_1 + p_12
print(f'x2:\n{x_2}')
print()

print('\033[1;33mSUBPROBLEM 3\033[0m')
print()
x_2 = x_2.flatten()
lamda_2 = np.array([-3.14256193, 0.06430786])  #lamda is associated with equality and inequality in previous problem
W_2 = hessian(f,x_2,lamda_2)
d_2 = gradient1(f, x_2)
A_eq_2 = gradient2(c_e, x_2)
b_eq_2 = np.array(-c_e(x_2, 0))
A_iq_2 = gradient2(c_i, x_2)
b_iq_2 = np.array(-c_i(x_2, 0))

print('Iteration 1')
print()
p_21 = np.array([[0.0],
                [0.0]])

A_k21 = np.vstack((A_eq_2, A_iq_2))
b_k21 = np.vstack((b_eq_2, b_iq_2))

q21, lamda21 = ECQP(W_2, d_2, A_k21, b_k21, p_21)

print(f'q21:\n {q21}')
print()
print(f'lamda21:\n {lamda21}')
print()
print(f'q21 norm:\n{np.linalg.norm(q21)}')
print()

p_22 = p_21 + q21 #alpha is 1 since no inactive working set

print('Iteration 1')
print()
A_k22 = np.vstack((A_eq_2, A_iq_2))
b_k22 = np.vstack((b_eq_2, b_iq_2))

q22, lamda22 = ECQP(W_2, d_2, A_k22, b_k22, p_22)

print(f'q22:\n {q22}')
print()
print(f'lamda22:\n {lamda22}')
print()
print(f'q22 norm:\n{np.linalg.norm(q22)}')
print()

x_2 = x_2.reshape(-1, 1)
x_3 = x_2 + p_22
print(f'x3:\n{x_3}')
print()

#stopping criteria
delx = (x_3 - x_2)
print(f'norm of delx = {np.linalg.norm(delx)}')
print()

print(f'\033[1;33mfinal answer:\n \033[0m \033[1;32m{x_3}\033[0m')




SUBPROBLEM 1

Iteration 1

q01:
 [[-2.3333333]
 [ 4.6666667]]

lamda01:
 [[-4.]]

q01 norm:
5.21749196240663

alpha_max: 
1

alpha_vec: 
[2.28571425]

Iteration 2

q02:
 [[-0.]
 [-0.]]

lamda02:
 [[-4.]]

q02 norm:
0.0

x1:
[[-1.3333333]
 [ 6.6666667]]

SUBPROBLEM 2

Iteration 1

q11:
 [[ 0.3143939]
 [-0.6287879]]

lamda11:
 [[-3.14256193]
 [ 0.06430786]]

q11 norm:
0.7030062215539916

Iteration 2

q12:
 [[ 0.]
 [-0.]]

lamda12:
 [[-3.14256193]
 [ 0.06430786]]

q12 norm:
0.0

x2:
[[-1.0189394]
 [ 6.0378788]]

SUBPROBLEM 3

Iteration 1

q21:
 [[ 0.0188709]
 [-0.0377418]]

lamda21:
 [[-3.07726102]
 [ 0.07681449]]

q21 norm:
0.042196615196600784

Iteration 1

q22:
 [[ 0.]
 [-0.]]

lamda22:
 [[-3.07726102]
 [ 0.07681449]]

q22 norm:
0.0

x3:
[[-1.0000685]
 [ 6.000137 ]]

norm of delx = 0.04219661519660083

final answer:
  [[-1.0000685]
 [ 6.000137 ]]


**Pentalty Function**

In [47]:
#taken from HW4

def line_search(f, x, d, alpha1=1.0, delta=0.1, ftol=1e-5, xtol=1e-5, max_iter=100):
  #line search using powell successive quadratic estimation

  g = lambda alpha: f(np.array(x) + alpha * np.array(d))  #converting multivariate function into univariate in alpha

  alpha2 = alpha1 + delta

  if g(alpha1) > g(alpha2):
    alpha3 = alpha1 + 2*delta
  else:
    alpha3 = alpha1 - delta

  for _ in range(max_iter):
    f1 = g(alpha1)
    f2 = g(alpha2)
    f3 = g(alpha3)

    f_min = min(f1, f2, f3)

    if f_min == f1:
      alpha_min = alpha1
    elif f_min == f2:
      alpha_min = alpha2
    else:
      alpha_min = alpha3

    a0 = f1
    a1 = (f2 - f1)/(alpha2-alpha1)
    a2 = ((f3 - f1)/(alpha3-alpha1) - (f2 - f1)/(alpha2 - alpha1))/(alpha3 - alpha2)

    alpha_star = (alpha1+alpha2)/2 - a1/(2*a2)

    if (g(alpha_star) - g(alpha_min)) < ftol and abs(alpha_star - alpha_min) < xtol: return alpha_star

    f_values = [[f1, alpha1], [f2, alpha2], [f3, alpha3], [g(alpha_star), alpha_star]]
    sorted_f = sorted(f_values, key=lambda x: x[0])
    y = [entry[1] for entry in sorted_f]
    alpha1, alpha2, alpha3 = y[0], y[1], y[2]

  return alpha_star

def powell_conjugate_method(f, x0, tol=1e-5, max_iter=100):
  n = len(x0)
  directions = np.eye(n)
  x = x0.copy()

  for iteration in range(max_iter):

      # Minimize along each direction
      for i in range(n):
          d = directions[i]
          alpha = line_search(f, x, d)
          x = x + alpha * d
          if i == 0:
            x_dash = x

      d_dash = directions[0]
      alpha_dash = line_search(f, x, d_dash)
      x = x + alpha_dash * d_dash


      d_new = x - x_dash

      if np.linalg.norm(d_new) < tol: return x

      d_new = (x - x_dash)/np.linalg.norm(x - x_dash)

      # Move directions to right and replace 0th direction with new direction
      directions = np.roll(directions, 1, axis=1)
      directions[0] = d_new

  return x

In [48]:
print('ITERATION 1, mu = 10')
def f1(x):
  return 3*x[0]**2 - 4*x[1] + (1/10)*(2*x[0] + x[1] - 4)**2 + (1/10)*(np.min((-x[0]**2 - x[1]**2 + 37), 0))**2    ##mu = 10
x0 = np.array([1.0, 2.0])
x1 = powell_conjugate_method(f1, x0)
print(f'Value after 1st iteration: \n {x1}')
print()

print('ITERATION 2, mu = 1')
def f2(x):
  return 3*x[0]**2 - 4*x[1] + (1/1)*(2*x[0] + x[1] - 4)**2 + (1/1)*(np.min((-x[0]**2 - x[1]**2 + 37), 0))**2    ##mu = 1
x2 = powell_conjugate_method(f2, x1)
print(f'Value after 2nd iteration: \n {x2}')
print()

print('ITERATION 3, mu = 0.1')
def f3(x):
  return 3*x[0]**2 - 4*x[1] + (1/0.1)*(2*x[0] + x[1] - 4)**2 + (1/0.1)*(np.min((-x[0]**2 - x[1]**2 + 37), 0))**2    ##mu = 0.1
x3 = powell_conjugate_method(f3, x2)
print(f'Value after 3rd iteration: \n {x3}')
print()

print('ITERATION 4, mu = 0.01')
def f4(x):
  return 3*x[0]**2 - 4*x[1] + (1/0.01)*(2*x[0] + x[1] - 4)**2 + (1/0.01)*(np.min((-x[0]**2 - x[1]**2 + 37), 0))**2    ##mu = 0.01
x4 = powell_conjugate_method(f4, x3)
print(f'Value after 4th iteration: \n {x4}')
print()

print('ITERATION 5, mu = 0.001')
def f5(x):
  return 3*x[0]**2 - 4*x[1] + (1/0.001)*(2*x[0] + x[1] - 4)**2 + (1/0.001)*(np.min((-x[0]**2 - x[1]**2 + 37), 0))**2    ##mu = 0.001
x5 = powell_conjugate_method(f5, x4)
print(f'Value after 5th iteration: \n {x5}')
print()

print('Value for x is not changing much after 4th iteration')
print()
print(f'\033[1;33mfinal answer:\n \033[0m \033[1;32m{x5}\033[0m')

ITERATION 1, mu = 10
Value after 1st iteration: 
 [-0.11921264  6.20003907]

ITERATION 2, mu = 1
Value after 2nd iteration: 
 [-0.57470118  6.06294621]

ITERATION 3, mu = 0.1
Value after 3rd iteration: 
 [-0.93338979  6.01110733]

ITERATION 4, mu = 0.01
Value after 4th iteration: 
 [-0.99294458  6.00120395]

ITERATION 5, mu = 0.001
Value after 5th iteration: 
 [-0.99930544  6.00011897]

Value for x is not changing much after 4th iteration

final answer:
  [-0.99930544  6.00011897]


**Barrier Function**

In [49]:
import math

def custom_log(x):
    if x > 0:
        return math.log(x)
    else:
        return -1000000 #very high negative value if constraint is violated


print('ITERATION 1, mu = 10')
def f1(x):
  return 3*x[0]**2 - 4*x[1] + (1/10)*(2*x[0] + x[1] - 4)**2 - 10*custom_log(-x[0]**2 - x[1]**2 + 37)   ##mu = 10
x0 = np.array([1.0, 2.0])
x1 = powell_conjugate_method(f1, x0)
print(f'Value after 1st iteration: \n {x1}')
print()

print('ITERATION 2, mu = 1')
def f2(x):
  return 3*x[0]**2 - 4*x[1] + (1/1)*(2*x[0] + x[1] - 4)**2 - 1*custom_log(-x[0]**2 - x[1]**2 + 37)     ##mu = 1
x2 = powell_conjugate_method(f2, x1)
print(f'Value after 2nd iteration: \n {x2}')
print()

print('ITERATION 3, mu = 0.1')
def f3(x):
  return 3*x[0]**2 - 4*x[1] + (1/0.1)*(2*x[0] + x[1] - 4)**2 - 0.1*custom_log(-x[0]**2 - x[1]**2 + 37)     ##mu = 0.1
x3 = powell_conjugate_method(f3, x2)
print(f'Value after 3rd iteration: \n {x3}')
print()

print('ITERATION 4, mu = 0.01')
def f4(x):
  return 3*x[0]**2 - 4*x[1] + (1/0.01)*(2*x[0] + x[1] - 4)**2 - 0.01*custom_log(-x[0]**2 - x[1]**2 + 37)     ##mu = 0.01
x4 = powell_conjugate_method(f4, x3)
print(f'Value after 4th iteration: \n {x4}')
print()

print('ITERATION 5, mu = 0.001')
def f5(x):
  return 3*x[0]**2 - 4*x[1] + (1/0.001)*(2*x[0] + x[1] - 4)**2 - 0.001*custom_log(-x[0]**2 - x[1]**2 + 37)    ##mu = 0.001
x5 = powell_conjugate_method(f5, x4)
print(f'Value after 5th iteration: \n {x5}')
print()

print('ITERATION 6, mu = 0.0001')
def f6(x):
  return 3*x[0]**2 - 4*x[1] + (1/0.0001)*(2*x[0] + x[1] - 4)**2 - 0.0001*custom_log(-x[0]**2 - x[1]**2 + 37)    ##mu = 0.001
x6 = powell_conjugate_method(f6, x5)
print(f'Value after 5th iteration: \n {x6}')
print()

print('Value for x is not changing much after 5th iteration')
print()
print(f'\033[1;33mFinal answer:\n \033[0m \033[1;32m{x6}\033[0m')
print()

print('Checking if Equality constraint is satisfied')
print(f'Final answer*A_e.T = {np.dot(x6, np.array([2, 1]))} = 4')
print('\033[1mEquality constraint is satisfied\033[0m')
print()

print('Checking if Inequality constraint is satisfied')
print(f'x1^2 + x2^2 = {x6[0]**2 + x6[1]**2}, not <= 37')
print('\033[1mInequality constraint is not satisfied\033[0m')

ITERATION 1, mu = 10
Value after 1st iteration: 
 [-3.67595471e-03  4.07149080e+00]

ITERATION 2, mu = 1
Value after 2nd iteration: 
 [-1.33333333  8.66666667]

ITERATION 3, mu = 0.1
Value after 3rd iteration: 
 [-1.1187328   6.42306212]

ITERATION 4, mu = 0.01
Value after 4th iteration: 
 [-1.33333333  6.68666667]

ITERATION 5, mu = 0.001
Value after 5th iteration: 
 [-1.33333333  6.66866667]

ITERATION 6, mu = 0.0001
Value after 5th iteration: 
 [-1.3342332   6.66866653]

Value for x is not changing much after 5th iteration

Final answer:
  [-1.3342332   6.66866653]

Checking if Equality constraint is satisfied
Final answer*A_e.T = 4.000200134979755 = 4
Equality constraint is satisfied

Checking if Inequality constraint is satisfied
x1^2 + x2^2 = 46.25129153828641, not <= 37
Inequality constraint is not satisfied
